<a href="https://www.kaggle.com/code/christianrogers1/simple-neural-network-for-the-titanic-challenge?scriptVersionId=225581041" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")
gender_submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

In [3]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [5]:
from sklearn.model_selection import train_test_split
train["age_estimated"] = (train["Age"] % 1 == 0.5).astype(int)
test["age_estimated"] = (test["Age"] % 1 == 0.5).astype(int)

train = train.replace({"Sex" : {
    "male" : 0,
    "female" : 1
}})
test = test.replace({"Sex" : {
    "male" : 0,
    "female" : 1
}})

print(train.isnull().sum())
train["Age"] = train["Age"].fillna(train["Age"].median())
test["Age"] = test["Age"].fillna(test["Age"].median())

X = train[["Pclass", "Sex", "Age", "Parch", "age_estimated"]]
y = train[["Survived"]]

test_data = test[["Pclass", "Sex", "Age", "Parch", "age_estimated"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

X_train.describe()

PassengerId        0
Survived           0
Pclass             0
Name               0
Sex                0
Age              177
SibSp              0
Parch              0
Ticket             0
Fare               0
Cabin            687
Embarked           2
age_estimated      0
dtype: int64


<ipython-input-5-41af27401f56>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train = train.replace({"Sex" : {
<ipython-input-5-41af27401f56>:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = test.replace({"Sex" : {


,Pclass,Sex,Age,Parch,age_estimated
count,712.000000,712.000000,712.000000,712.000000,712.000000
mean,2.330056,0.344101,29.204129,0.379213,0.019663
std,0.824584,0.475408,13.007971,0.791669,0.138937
min,1.000000,0.000000,0.420000,0.000000,0.000000
25%,2.000000,0.000000,22.000000,0.000000,0.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000
75%,3.000000,1.000000,35.000000,0.000000,0.000000
max,3.000000,1.000000,80.000000,6.000000,1.000000


In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [7]:
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [8]:
class LivingClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Linear(5, 10),
            nn.LeakyReLU(),
            
            nn.Linear(10, 10),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(10, 10),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(10, 10),
            nn.LeakyReLU(),
            nn.Linear(10, 1)
        )
    def forward(self, x):
        return self.stack(x)

model = LivingClassifier().to(device)
print(model)

LivingClassifier(
  (stack): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Dropout(p=0.2, inplace=False)
    (5): Linear(in_features=10, out_features=10, bias=True)
    (6): LeakyReLU(negative_slope=0.01)
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=10, out_features=10, bias=True)
    (9): LeakyReLU(negative_slope=0.01)
    (10): Linear(in_features=10, out_features=1, bias=True)
  )
)


In [9]:
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

X_train = torch.tensor(X_train.tolist())
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train.to_numpy())
y_test = torch.tensor(y_test.to_numpy())
#test_data = torch.tensor(test_data.to_numpy())

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [10]:
def test_accuracy():
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.float()
            outputs = model(inputs)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return f"Accuracy on test data: {100 * correct / total:.2f}%"

In [11]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)  # He initialization for ReLU
        nn.init.zeros_(m.bias)

model.apply(init_weights)

num_epochs = 30
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = torch.clamp(torch.sigmoid(model(inputs)), 1e-7, 1 - 1e-7)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f} ; " + test_accuracy())

Epoch [1/30], Loss: 0.6042 ; Accuracy on test data: 74.30%
Epoch [2/30], Loss: 0.5063 ; Accuracy on test data: 77.09%
Epoch [3/30], Loss: 0.5053 ; Accuracy on test data: 78.21%
Epoch [4/30], Loss: 0.4944 ; Accuracy on test data: 79.89%
Epoch [5/30], Loss: 0.4669 ; Accuracy on test data: 77.09%
Epoch [6/30], Loss: 0.4710 ; Accuracy on test data: 79.89%
Epoch [7/30], Loss: 0.4600 ; Accuracy on test data: 82.12%
Epoch [8/30], Loss: 0.4455 ; Accuracy on test data: 79.89%
Epoch [9/30], Loss: 0.4675 ; Accuracy on test data: 79.89%
Epoch [10/30], Loss: 0.4493 ; Accuracy on test data: 79.89%
Epoch [11/30], Loss: 0.4449 ; Accuracy on test data: 79.33%
Epoch [12/30], Loss: 0.4483 ; Accuracy on test data: 82.68%
Epoch [13/30], Loss: 0.4640 ; Accuracy on test data: 77.65%
Epoch [14/30], Loss: 0.4279 ; Accuracy on test data: 80.45%
Epoch [15/30], Loss: 0.4526 ; Accuracy on test data: 81.01%
Epoch [16/30], Loss: 0.4491 ; Accuracy on test data: 81.56%
Epoch [17/30], Loss: 0.4528 ; Accuracy on test da

In [12]:
predictions = []
ID = []
for i in range(892, 1310):
    ID.append(i)
def predict():
    with torch.no_grad():
        for i in range(len(test_data)):
            predictions.append((
                (
                    (
                        torch.sigmoid(
                            model(
                                torch.tensor(test_data.iloc[i].astype(float).values).float().to(device)
                            )
                        )
                    ).round()
                ).float()).cpu().numpy()
            )

In [13]:
predict()
result = pd.DataFrame(predictions)
result.insert(0, "ID", ID)

In [14]:
result.to_csv("nnPredict.csv", index=False)